<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/alpaca_LightGBM_execution_adapter_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install alpaca-trade-api
!pip install alpaca-trade-api python-dotenv ta

In [2]:
import os
import pickle
import csv
import pandas as pd
import lightgbm as lgb
import ta
import warnings
from datetime import datetime, timedelta
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST, TimeFrame, APIError

warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

RESULTS_DIR = "/content/drive/MyDrive/Alpaca_Results/LightGBM"
os.makedirs(RESULTS_DIR, exist_ok=True)

Mounted at /content/drive


In [3]:
from google.colab import files
uploaded = files.upload()  # Prompts you to upload the .env file for the key

Saving Alpaca_keys.env.txt to Alpaca_keys.env.txt


In [4]:
import os

# Rename to .env so dotenv can recognize it
os.rename("Alpaca_keys.env.txt", ".env")

In [5]:
from google.colab import files
uploaded = files.upload()  # Upload: model_ABT.txt, features_ABT.txt, scaler_ABT.pkl

import lightgbm as lgb
import pickle

# Load LightGBM model
model = lgb.Booster(model_file='model_ABT.txt')

# Load feature list
with open('features_ABT.txt', 'r') as f:
    model_features = f.read().strip().split(',')


# Load scaler
with open('scaler_ABT.pkl', 'rb') as f:
    scaler = pickle.load(f)

print("Model, features, and scaler loaded for ABT.")


Saving features_ABT.txt to features_ABT (3).txt
Saving features_BRK-B.txt to features_BRK-B (3).txt
Saving features_MDT.txt to features_MDT (3).txt
Saving features_PG.txt to features_PG (3).txt
Saving features_PM.txt to features_PM (3).txt
Saving model_ABT.txt to model_ABT (2).txt
Saving model_BRK-B.txt to model_BRK-B (3).txt
Saving model_MDT.txt to model_MDT (3).txt
Saving model_PG.txt to model_PG (3).txt
Saving model_PM.txt to model_PM (3).txt
Saving scaler_ABT.pkl to scaler_ABT (3).pkl
Saving scaler_BRK-B.pkl to scaler_BRK-B (3).pkl
Saving scaler_MDT.pkl to scaler_MDT (3).pkl
Saving scaler_PG.pkl to scaler_PG (3).pkl
Saving scaler_PM.pkl to scaler_PM (3).pkl
Model, features, and scaler loaded for ABT.


In [6]:
# Load .env credentials
load_dotenv(override=True)
API_KEY = os.getenv("APCA_API_KEY_ID")
SECRET_KEY = os.getenv("APCA_API_SECRET_KEY")
BASE_URL = "https://paper-api.alpaca.markets"  # or live URL

# Create API object
api = REST(API_KEY, SECRET_KEY, BASE_URL, api_version='v2')

# Check account
account = api.get_account()
print("Account status:", account.status)


Account status: ACTIVE


In [7]:
#Load Model, Scaler, Features Dynamically
def load_model_artifacts(ticker):
    model = lgb.Booster(model_file=f'model_{ticker}.txt')
    with open(f'features_{ticker}.txt', 'r') as f:
        features = f.read().strip().split(',')
    with open(f'scaler_{ticker}.pkl', 'rb') as f:
        scaler = pickle.load(f)
    return model, features, scaler

In [8]:
def add_features(df):
    """Add engineered technical features to the DataFrame."""
    df['SMA_50'] = ta.trend.sma_indicator(df['close'], window=50)
    df['EMA_20'] = ta.trend.ema_indicator(df['close'], window=20)
    volatility = df['close'].rolling(14).std()
    rsi = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
    macd = ta.trend.macd_diff(df['close'])
    signal_line = ta.trend.macd_signal(df['close'])
    df['RSI_voladj'] = rsi * volatility
    df['MACD_voladj'] = macd * volatility
    df['Signal_Line'] = signal_line
    df['ATR_voladj'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=14).average_true_range() * volatility
    df['OBV'] = ta.volume.on_balance_volume(df['close'], df['volume'])
    df['CCI_voladj'] = ta.trend.cci(df['high'], df['low'], df['close'], window=20) * volatility
    return df.dropna()

In [9]:
def log_trade(ticker, signal, action, price, equity, position_qty):
    now = datetime.utcnow().isoformat()
    row = {
        "datetime": now,
        "ticker": ticker,
        "signal": signal,
        "action": action,
        "price": price,
        "equity": equity,
        "position_qty": position_qty
    }
    log_file = os.path.join(RESULTS_DIR, f"trade_log_{ticker}.csv")
    file_exists = os.path.isfile(log_file)
    with open(log_file, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=row.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)
    print(f"📒 Logged trade to Drive: {log_file}")


In [10]:
tickers = ["ABT", "BRK.B", "MDT", "PG", "PM"]

for ticker in tickers:
    print(f"\nRunning strategy for {ticker}...")

    # Load model artifacts
    try:
        model, model_features, scaler = load_model_artifacts(ticker)
        print(f"Model artifacts loaded for {ticker}")
    except Exception as e:
        print(f"Could not load model for {ticker}: {e}")
        continue

    # Fetch bars
    end = datetime.utcnow().replace(microsecond=0).isoformat() + "Z"
    start = (datetime.utcnow() - timedelta(days=45)).replace(microsecond=0).isoformat() + "Z"
    try:
        bars = api.get_bars(ticker, TimeFrame.Hour, start=start, end=end, feed="iex").df
        bars = bars.reset_index()
        if len(bars) < 100:
            print(f"Not enough data for {ticker}: {len(bars)} rows")
            continue
        bars = add_features(bars)
        if len(bars) < 10:
            print(f"Not enough usable rows after feature engineering for {ticker}")
            continue
    except Exception as e:
        print(f"Error fetching/processing bars for {ticker}: {e}")
        continue

    # Check features
    if not all(f in bars.columns for f in model_features):
        missing = [f for f in model_features if f not in bars.columns]
        print(f"Missing features for {ticker}: {missing}")
        continue

    # Predict
    X_live = bars[model_features].copy().dropna()
    X_scaled = scaler.transform(X_live)
    preds = model.predict(X_scaled)
    signal = int(preds[-1])
    print(f"Prediction for {ticker}: {signal} (1 = Buy, 0 = Sell)")

    # Trade execution
    price = bars.iloc[-1]["close"]
    history = api.get_portfolio_history(period="1D", timeframe="5Min")
    equity = history.equity[-1]
    action = "Hold"
    position_qty = 0

    try:
        clock = api.get_clock()
        if not clock.is_open:
            print("Market is closed.")
            continue

        try:
            position = api.get_position(ticker)
            has_position = float(position.qty) > 0
            position_qty = float(position.qty)
        except APIError:
            has_position = False
            position_qty = 0
            print(f"ℹNo current position in {ticker}")

        if signal == 1 and not has_position:
            api.submit_order(symbol=ticker, qty=1, side='buy', type='market', time_in_force='gtc')
            action = "Buy"
            print(f"BUY order submitted for {ticker}")

        elif signal == 0 and has_position:
            api.submit_order(symbol=ticker, qty=1, side='sell', type='market', time_in_force='gtc')
            action = "Sell"
            print(f"SELL order submitted for {ticker}")

        else:
            print(f"No action taken for {ticker}")

    except Exception as e:
        print(f"Trade error for {ticker}: {e}")
        continue

    # Log trade
    log_trade(ticker, signal, action, price, equity, position_qty)

    # Optional: display portfolio snapshot
    print("Portfolio snapshot:")
    positions = api.list_positions()
    for p in positions:
        print(f"{p.symbol}: {p.qty} shares (${p.market_value})")

    print("Recent Equity:", history.equity[-3:])
    print(f"Strategy complete for {ticker}")


Running strategy for ABT...
Model artifacts loaded for ABT
Prediction for ABT: 0 (1 = Buy, 0 = Sell)
Market is closed.

Running strategy for BRK.B...
Could not load model for BRK.B: Could not open model_BRK.B.txt

Running strategy for MDT...
Model artifacts loaded for MDT
Prediction for MDT: 0 (1 = Buy, 0 = Sell)
Market is closed.

Running strategy for PG...
Model artifacts loaded for PG
Prediction for PG: 0 (1 = Buy, 0 = Sell)
Market is closed.

Running strategy for PM...
Model artifacts loaded for PM
Prediction for PM: 0 (1 = Buy, 0 = Sell)
Market is closed.
